In [25]:
import plotly.express as px
import pandas as pd
import re

In [26]:
DATA_INPUT = '/home/re1jie/TA-code/JSSP-CAOA-SSR/Data/raw_data.csv'
df = pd.read_csv(DATA_INPUT)
df.head()

,NAMA_KAPAL,VOYAGE,PELABUHAN,ETA_HARI,ETA_TANGGAL,ETA_JAM,ETD_HARI,ETD_TANGGAL,ETD_JAM
0,KM.AWU,1.2025,Kumai,Rabu,8-Jan-25,04:00,Rabu,8-Jan-25,14:00
1,KM.AWU,1.2025,Surabaya,Kamis,9-Jan-25,17:00,Kamis,9-Jan-25,23:59
2,KM.AWU,1.2025,Benoa,Sabtu,11-Jan-25,03:00,Sabtu,11-Jan-25,09:00
3,KM.AWU,1.2025,Bima,Minggu,12-Jan-25,06:00,Minggu,12-Jan-25,08:00
4,KM.AWU,1.2025,Waingapu,Minggu,12-Jan-25,22:00,Senin,13-Jan-25,01:00


## Normalisasi datetime

In [27]:
# Mapping helper
bulan_map = {
    "Jan": "Jan",
    "Feb": "Feb",
    "Mar": "Mar",
    "Apr": "Apr",
    "Mei": "May",
    "Jun": "Jun",
    "Jul": "Jul",
    "Agu": "Aug",
    "Sep": "Sep",
    "Okt": "Oct",
    "Nov": "Nov",
    "Des": "Dec"
}

pattern = re.compile("|".join(bulan_map.keys()))

def normalize_date(date_str):
    if pd.isna(date_str):
        return None

    date_str = str(date_str)

    return pattern.sub(lambda x: bulan_map[x.group()], date_str)

df["ETA_TANGGAL"] = df["ETA_TANGGAL"].apply(normalize_date)
df["ETD_TANGGAL"] = df["ETD_TANGGAL"].apply(normalize_date)

# Build datetime ETA
df["ETA"] = pd.to_datetime(
    df["ETA_TANGGAL"] + " " + df["ETA_JAM"],
    format="%d-%b-%y %H:%M",
    errors="coerce"
)

df["ETD"] = pd.to_datetime(
    df["ETD_TANGGAL"] + " " + df["ETD_JAM"],
    format="%d-%b-%y %H:%M",
    errors="coerce"
)

df = df.sort_values(
    ["NAMA_KAPAL", "ETA"]
)

df["ETD"] = df["ETD"].fillna(
    df.groupby("NAMA_KAPAL")["ETA"].shift(-1)
)

invalid = df[df["ETD"] < df["ETA"]]
print("Invalid rows:", len(invalid))

df.head()

Invalid rows: 0


,NAMA_KAPAL,VOYAGE,PELABUHAN,ETA_HARI,ETA_TANGGAL,ETA_JAM,ETD_HARI,ETD_TANGGAL,ETD_JAM,ETA,ETD
3558,KFC.JETLINER,1.2025,Kendari,Sabtu,28-Dec-24,22:00,Minggu,29-Dec-24,07:00,2024-12-28 22:00:00,2024-12-29 07:00:00
3559,KFC.JETLINER,1.2025,Wanci,Minggu,29-Dec-24,20:00,Minggu,29-Dec-24,22:00,2024-12-29 20:00:00,2024-12-29 22:00:00
3560,KFC.JETLINER,1.2025,Kendari,Senin,30-Dec-24,11:00,Senin,30-Dec-24,17:00,2024-12-30 11:00:00,2024-12-30 17:00:00
3561,KFC.JETLINER,1.2025,Wanci,Selasa,31-Dec-24,06:00,Selasa,31-Dec-24,08:00,2024-12-31 06:00:00,2024-12-31 08:00:00
3562,KFC.JETLINER,1.2025,Kendari,Selasa,31-Dec-24,21:00,Selasa,31-Dec-24,23:00,2024-12-31 21:00:00,2024-12-31 23:00:00


In [33]:
print(df[["NAMA_KAPAL","VOYAGE","PELABUHAN","ETA","ETD"]].head(20))

        NAMA_KAPAL  VOYAGE PELABUHAN                 ETA                 ETD
3558  KFC.JETLINER  1.2025   Kendari 2024-12-28 22:00:00 2024-12-29 07:00:00
3559  KFC.JETLINER  1.2025     Wanci 2024-12-29 20:00:00 2024-12-29 22:00:00
3560  KFC.JETLINER  1.2025   Kendari 2024-12-30 11:00:00 2024-12-30 17:00:00
3561  KFC.JETLINER  1.2025     Wanci 2024-12-31 06:00:00 2024-12-31 08:00:00
3562  KFC.JETLINER  1.2025   Kendari 2024-12-31 21:00:00 2024-12-31 23:00:00
3563  KFC.JETLINER  1.2025      Raha 2025-01-01 07:00:00 2025-01-01 08:00:00
3564  KFC.JETLINER  1.2025   Kendari 2025-01-01 16:00:00 2025-01-01 18:00:00
3565  KFC.JETLINER  1.2025     Wanci 2025-01-02 07:00:00 2025-01-02 09:00:00
3566  KFC.JETLINER  1.2025   Kendari 2025-01-02 22:00:00 2025-01-03 17:00:00
3567  KFC.JETLINER  1.2025     Wanci 2025-01-04 06:00:00 2025-01-04 08:00:00
3568  KFC.JETLINER  1.2025   Kendari 2025-01-04 21:00:00 2025-01-04 21:00:00
3569  KFC.JETLINER  2.2025   Kendari 2025-01-04 21:00:00 2025-01-05 07:00:00

## Visualisasi

In [34]:
df = df.sort_values(["NAMA_KAPAL", "ETA"]).reset_index(drop=True)

df["NEXT_ETA"] = df.groupby("NAMA_KAPAL")["ETA"].shift(-1)

sandar = df.copy()

sandar["START"] = sandar["ETA"]
sandar["END"] = sandar["ETD"]
sandar["STATUS"] = "Sandar"

layar = df.copy()

layar["START"] = layar["ETD"]
layar["END"] = layar["NEXT_ETA"]
layar["STATUS"] = "Layar"

layar = layar.dropna(subset=["START", "END"])

timeline = pd.concat([sandar, layar], ignore_index=True)

timeline = timeline[timeline["END"] > timeline["START"]]

In [ ]:
fig = px.timeline(
    timeline,
    x_start="START",
    x_end="END",
    y="NAMA_KAPAL",
    color="STATUS",            # Sandar vs Layar
    hover_data=[
        "VOYAGE",
        "PELABUHAN",
        "ETA",
        "ETD"
    ]
)

fig.update_yaxes(autorange="reversed")

fig.update_layout(
    title="Timeline Operasional Kapal (Sandar vs Layar)",
    xaxis_title="Waktu",
    yaxis_title="Kapal",
    hovermode="closest"
)

fig.show()

ValueError: Value of 'hover_data_1' is not the name of a column in 'data_frame'. Expected one of ['NAMA_KAPAL', 'VOYAGE', 'PELABUHAN', 'ETA_HARI', 'ETA_TANGGAL', 'ETA_JAM', 'ETD_HARI', 'ETD_TANGGAL', 'ETD_JAM', 'ETA', 'ETD', 'NEXT_ETA', 'START', 'END', 'STATUS'] but received: PELAPUHAN